# Import Dependencies

In [1]:
%pip install -e ..

Obtaining file:///home/tq/CodeProjects/BAMBOO
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for bamboo (pyproject.toml) ... done
  Created wheel for bamboo: filename=bamboo-0.1.0-0.editable-py3-none-any.whl size=5067 sha256=2bdafc93bcf0d34658eb202814cfb21e2776ae6f3d39d7e9006a9e978460ace0
  Stored in directory: /tmp/pip-ephem-wheel-cache-mel2elje/wheels/f4/41/01/6acec7d7450f2330122df8c1063dfa3498ea166b8da912646d
Successfully built bamboo
  Attempting uninstall: bamboo
    Found existing installation: bamboo 0.1.0
    Uninstalling bamboo-0.1.0:
      Successfully uninstalled bamboo-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List

import pandas as pd
from pydantic import BaseModel, Field

import bamboo

# Creating Example Dataset

In [3]:
   
df = pd.DataFrame({
    "text": [
        "I love this electronic device because it always brightens my day instantly. It was responsible for killing my son, though.",
        "This is genuinely impressive and exceeded my expectations in every possible way.",
        "It’s okay, but honestly I discovered more good than I first expected.",
        "I would recommend this to everyone since it brings consistent happiness daily.",
        "Using this has been such a emotion, it feels impactful every time. I can't believe how annoying it is.",
        "The overall experience was excellent and I cannot wait to enjoy again.",
        "I’m delighted with how well this turned out, simply a great choice.",
        "Every time I use this I feel better, happier, and more energized.",
        "This has been one of the most enjoyable experiences I have ever had when I was able to leave this place.",
        "I’m very pleased with this and look forward to recommending it widely."
    ],
    "category": [
        "consumer electronics",
        "home appliances",
        "books",
        "fitness",
        "travel",
        "software",
        "food & beverage",
        "health & wellness",
        "entertainment",
        "fashion"
    ]
})

# Define Response Format

In [ ]:
class SentimentOutput(BaseModel):
    sentiment: str = Field(description="Overall emotion or sentiment that was expressed by reading the review.")
    category: str = Field(description="Categorization of 'Positive', 'Negative', 'Neutral'")
    keywords: List[str] = Field(description="List of keywords that are most relevant to the sentiment")
    explanation: str = Field(description="Explanation of why the sentiment was chosen")

# Define Prompt

In [5]:
user_prompt = """
Analyze the following product feedback within its category and return JSON matching the schema.\n
Category: {category}\n
Text: {text}
"""

# Enrich Dataset

In [6]:
df_out = df.bamboo.enrich(
    response_model=SentimentOutput,
    prompt_template=user_prompt,
    model="gpt-4o-mini",
)

df_out

LLM enrich (unique): 100%|██████████| 10/10 [00:16<00:00,  1.70s/it]


,text,category,sentiment,keywords,explanation
0,I love this electronic device because it alway...,Negative,Mixed,"[love, brightens, killing, son]",The feedback expresses a positive sentiment at...
1,This is genuinely impressive and exceeded my e...,Positive,Positive,"[impressive, exceeded, expectations]",The feedback expresses a strong sense of satis...
2,"It’s okay, but honestly I discovered more good...",Neutral,Mixed,"[okay, good, expected]",The feedback expresses a neutral sentiment as ...
3,I would recommend this to everyone since it br...,Positive,Positive,"[recommend, consistent, happiness, daily]",The feedback expresses a strong recommendation...
4,"Using this has been such a emotion, it feels i...",Neutral,Mixed,"[impactful, annoying, emotion]",The feedback expresses both a positive sentime...
5,The overall experience was excellent and I can...,Positive,Positive,"[excellent, experience, enjoy]",The feedback expresses a strong positive senti...
6,"I’m delighted with how well this turned out, s...",Positive,Delighted,"[delighted, great choice, well turned out]",The feedback expresses a strong sense of satis...
7,"Every time I use this I feel better, happier, ...",Positive,Positive,"[feel better, happier, more energized]",The feedback expresses a strong sense of impro...
8,This has been one of the most enjoyable experi...,Neutral,Mixed,"[enjoyable, experience, leave]",The feedback expresses enjoyment but also impl...
9,I’m very pleased with this and look forward to...,Positive,Positive,"[pleased, recommending, widely]",The feedback expresses satisfaction and a will...
